## 5. Thread‑safety tools: Lock, RLock, Semaphore, Queue

**Lock** (mutex) – only one thread enters critical section.

**RLock** – re‑entrant lock; same thread can acquire multiple times (needed in recursive callbacks).

**Semaphore(n)** – up to *n* threads enter (connection pools, rate limiters).

**queue.Queue** – built‑in locking FIFO; pass data instead of sharing vars → *share‑nothing, communicate* pattern.

```python
import threading, queue, time
counter = 0
lock = threading.Lock()
def safe_inc():
    global counter
    for _ in range(100_000):
        with lock:
            counter += 1

threads=[threading.Thread(target=safe_inc) for _ in range(2)]
for t in threads:t.start()
for t in threads:t.join()
print('counter', counter)

# queue demo
q = queue.Queue()
def producer():
    for i in range(3): q.put(i)
    q.put(None)  # poison pill
def consumer():
    while (item := q.get()) is not None:
        print('got', item); q.task_done()
threading.Thread(target=producer).start()
threading.Thread(target=consumer).start()
```

### Quick check

1. `with lock:` is equivalent to calling:
  a. `lock.acquire()` then `lock.release()`  b. `lock.wait()`

2. True / False `Queue.put()` blocks if size limit reached.

<details><summary>Answer key</summary>

1. **a**.
2. **True** – unless `block=False`.

</details>

## 6. Multiprocessing primer: sidestepping the GIL

`multiprocessing` spawns *processes* → each has its own Python interpreter and memory, so no GIL contention.  IPC via pipes/queues (pickle under the hood).  Ideal for CPU‑bound jobs.

ProcessPoolExecutor abstracts pool management; beware start‑up cost on Windows (spawn).

```python
from concurrent.futures import ProcessPoolExecutor
def fib(n):
    a,b=0,1
    for _ in range(n): a,b=b,a+b
    return a

with ProcessPoolExecutor() as ex:
    print(list(ex.map(fib, [30]*4)))
```

### Quick check

1. GIL is shared between processes?
  a. yes   b. no

2. True / False `multiprocessing` serialises arguments with pickle.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 7. Event loop & cooperative multitasking

`asyncio` runs a **single thread** event loop; coroutines volunteer control by `await`.  No pre‑emption → no race on Python objects, but you must **avoid blocking** calls.

Analogy: kids on a trampoline (event loop) jump one at time but switch quickly when they land.

```python
import asyncio, time
async def hello(n):
    await asyncio.sleep(1)
    print('hello', n)

t0=time.perf_counter()
asyncio.run(asyncio.gather(*(hello(i) for i in range(3))))
print('elapsed', time.perf_counter()-t0)
```

### Quick check

1. Event loop can run on multiple OS threads by default?
  a. yes   b. no

2. True / False Blocking `time.sleep(1)` inside coroutine freezes entire loop.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 8. Coroutines, `await`, `async def`

`async def` declares coroutine function; calling it returns **coroutine object**.  `await` suspends until awaited object resolves.  `asyncio.cancel()` raises `CancelledError` inside coroutine → add `try/except` for cleanup.

```python
import asyncio
async def work():
    await asyncio.sleep(0.5)
    return 42

async def main():
    coro = work()
    print('type', type(coro))
    result = await coro
    print('result', result)
asyncio.run(main())
```

### Quick check

1. `async def f()` when called returns:
  a. value   b. coroutine

2. True / False You can use `await` inside ordinary `def`.

<details><summary>Answer key</summary>

1. **b**.
2. **False**.

</details>